In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset = dataset.drop("User ID",axis=1)
dataset = pd.get_dummies(dataset,drop_first=True)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [4]:
 dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [5]:
Indep = dataset [['Age', 'EstimatedSalary', 'Gender_Male']]
dep = dataset [['Purchased']]

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(Indep,dep,test_size=1/3,random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

grid_param = {'criterion' :['gini', 'entropy', 'log_loss'], 'max_features':['sqrt', 'log2', None]}

grid = GridSearchCV(RandomForestClassifier(),grid_param,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
                    
grid.fit(x_train,y_train)


Fitting 5 folds for each of 9 candidates, totalling 45 fits


C:\Users\YAS\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\YAS\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\YAS\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 458, in fit
    for i, t in enumerate(trees)
  File "C:\Users\YAS\Anaconda3\lib\site-packages\joblib\parallel.py", line 921, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\YAS\Anaconda3\lib\site-packages\jo

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None]},
             scoring='f1_weighted', verbose=3)

In [10]:
y_pred = grid.predict(x_test)

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix (y_test,y_pred)
print(cm)

[[79  6]
 [ 4 45]]


In [12]:
from sklearn.metrics import classification_report
class_report = classification_report(y_test,y_pred)
print(class_report)

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        85
           1       0.88      0.92      0.90        49

    accuracy                           0.93       134
   macro avg       0.92      0.92      0.92       134
weighted avg       0.93      0.93      0.93       134



In [13]:
re= grid.cv_results_

In [14]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test,y_pred,average='micro')

print("The F1 value best score {}".format(f1),grid.best_params_)

The F1 value best score 0.9253731343283582 {'criterion': 'entropy', 'max_features': 'log2'}


In [15]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9674669867947179

In [16]:
result = pd.DataFrame.from_dict(re)
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.116342,0.003853,0.014532,0.003907,gini,sqrt,"{'criterion': 'gini', 'max_features': 'sqrt'}",0.867478,0.907177,0.851527,0.925272,0.962264,0.902744,0.039841,2
1,0.117152,0.003677,0.015701,0.003903,gini,log2,"{'criterion': 'gini', 'max_features': 'log2'}",0.847141,0.889022,0.833323,0.925272,0.943699,0.887692,0.042781,4
2,0.119069,0.000832,0.011673,0.000807,gini,None,"{'criterion': 'gini', 'max_features': None}",0.826263,0.849057,0.851527,0.869709,0.924528,0.864217,0.033163,6
3,0.115396,0.004321,0.013266,0.003933,entropy,sqrt,"{'criterion': 'entropy', 'max_features': 'sqrt'}",0.847141,0.889022,0.851527,0.925272,0.962264,0.895045,0.043947,3
4,0.115925,0.003220,0.015272,0.003505,entropy,log2,"{'criterion': 'entropy', 'max_features': 'log2'}",0.887346,0.907177,0.851527,0.925272,0.943699,0.903004,0.031826,1
5,0.126975,0.003679,0.012465,0.002948,entropy,None,"{'criterion': 'entropy', 'max_features': None}",0.847141,0.868752,0.851527,0.887907,0.924528,0.875971,0.028217,5
6,0.059796,0.005083,0.000000,0.000000,log_loss,sqrt,"{'criterion': 'log_loss', 'max_features': 'sqrt'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
7,0.060900,0.000803,0.000000,0.000000,log_loss,log2,"{'criterion': 'log_loss', 'max_features': 'log2'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
8,0.049870,0.011892,0.000000,0.000000,log_loss,None,"{'criterion': 'log_loss', 'max_features': None}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9


In [32]:
Age = int(input("Enter Age :"))
Sal = int(input("Enter Salary :"))
Gender = int(input("Enter either 0 or 1 :"))

Result = grid.predict([[Age,Sal,Gender]])

if Result == 0:
    print("Based on the input user NOT PURCHASE")
else:
    print("Based on the input user PURCHASE")

Enter Age :19
Enter Salary :76000
Enter either 0 or 1 :0
Based on the input user NOT PURCHASE


C:\Users\YAS\Anaconda3\lib\site-packages\sklearn\base.py:442: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
